In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls -al /kaggle/input/landmark-recognition-2020

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/kaggle/input/landmark-recognition-2020/train.csv")
df.head()

In [ ]:
print(len(df))
df = df[df['landmark_id'].map(df['landmark_id'].value_counts())==99]
print(len(df))
category_count = len(df.landmark_id.unique())
print(category_count)

In [ ]:
df.landmark_id = df.landmark_id.astype('category')
df['class_id'] = df.landmark_id.cat.codes

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
plt.hist(df.class_id, bins=1000)
plt.show()

In [ ]:
image_ids = df.id.to_numpy()
print(image_ids.shape)
print(image_ids[:3])

class_id = df.class_id.to_numpy()
print(class_id.shape)
print(class_id[:3])

In [ ]:
from PIL import Image

def load_image(image_id):
    full_path = "/kaggle/input/landmark-recognition-2020/{}/{}/{}/{}/{}.jpg".format(
        "train", image_id[0], image_id[1], image_id[2], image_id)

    img = Image.open(full_path)
    image = img.resize((224, 224))
    return np.array(image)

image = load_image("17660ef415d37059")
plt.imshow(image)
plt.show()

In [ ]:
x = []
y = []
print("total {} images.".format(len(image_ids)))
for i, image_id in enumerate(image_ids):
    if i%100==99:
        print("{} images loaded".format(i+1))
    x.append(load_image(image_id))
    y.append(class_id[i])

In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
i = int(len(x)*0.8)

train_x, test_x = x[:i], x[i:]
train_y, test_y = y[:i], y[i:]

In [ ]:
for i in range(10):
    plt.imshow(train_x[i])
    plt.show()

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np

from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

from tensorflow.keras.applications import efficientnet
from tensorflow.keras.applications import EfficientNetB2

preprocessor = efficientnet.preprocess_input

conv_layers = EfficientNetB2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
conv_layers.trainable = False

model = keras.Sequential()

model.add(conv_layers)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(category_count, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop", metrics=['acc'])
model.summary()

BATCH_SIZE = 64

train_data_generator = ImageDataGenerator(
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      vertical_flip=False,
      preprocessing_function=preprocessor
).flow(train_x, train_y, batch_size=BATCH_SIZE)

test_data_generator = ImageDataGenerator(
      preprocessing_function=preprocessor
).flow(test_x, test_y, batch_size=BATCH_SIZE)


history = model.fit(
      train_data_generator,
      validation_data=test_data_generator,
      epochs=1
)


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()


loss, acc = model.evaluate(test_data_generator)
print("loss=", loss)
print("acc=", acc)



test_x, test_y = test_data_generator.next()
y_ = model.predict(test_x)
predicted = np.argmax(y_, axis=-1)

plt.plot(test_y[:100], "o")
plt.plot(predicted[:100], '.')
plt.show()

for i in range(5):
    plt.imshow(test_x[i].astype(np.int))
    plt.show()
    print("label category index =", test_y[i])
    print("predicted category index =", predicted[i])